# Initial

In [95]:
from pynq_dpu import DpuOverlay
overlay = DpuOverlay('dpu_uartlite.bit')

from time import sleep, time

RX_FIFO = 0x00
TX_FIFO = 0x04

#Status Reg
STAT_REG = 0x08
RX_VALID = 0
RX_FULL = 1
TX_EMPTY = 2
TX_FULL = 3
IS_INTR = 4
OVERRUN_ERR = 5
FRAME_ERR = 6
PARITY_ERR =7

#Ctrl Reg
CTRL_REG = 0x0C
RST_TX = 0
RST_RX = 1
INTR_EN = 4

class UartLite:
    def __init__(self, uart):
        self.uart = uart
        self.setupCtrlReg()

    def getBit(self,num,pos):
        return (num&1<<pos)>>pos

    def setupCtrlReg(self):
        # Reset FIFOs, disable interrupts
        self.uart.write(CTRL_REG, 1<<RST_TX | 1<<RST_RX)
        sleep(0.5)
        self.uart.write(CTRL_REG, 0)
        sleep(0.5)

    def status(self):
        """Returns object that specifies current status of axi core"""
        status = self.uart.read(STAT_REG)
        return {'RX_VALID':self.getBit(status,RX_VALID),
            'RX_FULL':self.getBit(status, RX_FULL),
            'TX_EMPTY':self.getBit(status, TX_EMPTY),
            'TX_FULL':self.getBit(status, TX_FULL),
            'IS_INTR':self.getBit(status, IS_INTR),
            'OVERRUN_ERR':self.getBit(status, OVERRUN_ERR),
            'FRAME_ERR':self.getBit(status, FRAME_ERR),
            'PARITY_ERR':self.getBit(status, PARITY_ERR)}

    def read(self, count, timeout=10):
        buf = ""
        stop_time = time() + timeout
        for i in range(count):
            while (not (self.uart.read(STAT_REG) & 1<<RX_VALID)) and (time()<stop_time):
                pass
            if time() >= stop_time:
                break
            buf += chr(self.uart.read(RX_FIFO))
        return buf

    def write(self, buf, timeout=10):
        stop_time = time() + timeout
        wr_count = 0
        for i in buf:
            while (self.uart.read(STAT_REG) & 1<<TX_FULL) and (time()<stop_time):
                pass
            if time() > stop_time:
                break
            self.uart.write(TX_FIFO, ord(i))
            wr_count += 1
        return wr_count   

    def readline(self):
        ret = ""
        buf = self.read(1)
        if len(buf) == 0:
            return ""
        while buf != '\n':
            ret += buf
            buf = self.read(1)
        return ret
    
uart = UartLite(overlay.axi_uartlite_0)

print("UART overlay done")

UART overlay done


# UART package example
### right back 255, left forward 255

In [34]:
uart.write("02551255\n")

9

### right back 127, left back 127

In [96]:
uart.write("01270127\n")

9

### right forward 0, left forward 0 (stop)

In [97]:
uart.write("10001000\n")

9

# Use Twist to control

In [98]:
def cmd_vel_control(linear_x, angular_z):
    WHEEL_DIST = 5;
    speed_wish_right = (angular_z * WHEEL_DIST)/2 + linear_x
    speed_wish_left = linear_x * 2-speed_wish_right
    
    speed_wish_right = int(255*speed_wish_right)
    speed_wish_left = int(255*speed_wish_left)
    if speed_wish_right > 255: speed_wish_right = 255
    if speed_wish_right < -255: speed_wish_right = -255
    if speed_wish_left > 255: speed_wish_left = 255
    if speed_wish_left < -255: speed_wish_left = -255
    print("speed_wish_right, speed_wish_left= ",speed_wish_right, ",",speed_wish_left)
    
    
    pkg="1" if speed_wish_right>=0 else "0"
    speed_wish_right = abs(speed_wish_right)
    pkg += str(speed_wish_right//100)
    pkg += str((speed_wish_right%100)//10)
    pkg += str(speed_wish_right%10)
    
    if speed_wish_left>=0: pkg += "1"
    else : pkg += "0"
    speed_wish_left = abs(speed_wish_left)
    pkg += str(speed_wish_left//100)
    pkg += str((speed_wish_left%100)//10)
    pkg += str(speed_wish_left%10)
    pkg += "\n"
    
    print("uart pkg= ", pkg)
    
    uart.write(pkg)

In [102]:
cmd_vel_control(0, 0)

speed_wish_right, speed_wish_left=  0 , 0
uart pkg=  10001000



In [100]:
cmd_vel_control(0.5, 0)

speed_wish_right, speed_wish_left=  127 , 127
uart pkg=  11271127



In [101]:
cmd_vel_control(0, -0.5)

speed_wish_right, speed_wish_left=  -255 , 255
uart pkg=  02551255

